# Twitter API Workbook

In [3]:
import pandas as pd
import json
import re
import requests
import pickle
import config
from utils import *

## Database 1

In [4]:
df = pd.read_csv('../data/hate.csv')
df.head()

,Tweet ID,User ID,Hate Probability,Counterhate Probability,Neutral Probability,Label
0,1242513753733808128,1037398540698312704,0.625769,0.210356,0.063975,Hate
1,1242355882119155713,1242352444136247298,0.793170,0.133605,0.008983,Hate
2,1242280806711611392,14369596,0.664656,0.447038,0.075040,Hate
3,1242484533875707910,1067189627721850880,0.798824,0.015304,0.112250,Hate
4,1242433526407917568,3139806542,0.969103,0.001253,0.092968,Hate


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891204 entries, 0 to 891203
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Tweet ID                 891204 non-null  int64  
 1   User ID                  891204 non-null  int64  
 2   Hate Probability         891204 non-null  float64
 3   Counterhate Probability  891204 non-null  float64
 4   Neutral Probability      891204 non-null  float64
 5   Label                    891204 non-null  object 
dtypes: float64(3), int64(2), object(1)
memory usage: 40.8+ MB


In [23]:
sorted_df = df.sort_values(by='Hate Probability', ascending=False).reset_index()
sorted_df.columns= ['index', 'id', 'user', 'hate', 'chate', 'neutral', 'label']
sorted_df.head()

,index,id,user,hate,chate,neutral,label
0,560314,1240976182494908416,884055068265164801,0.999974,0.000520,0.000382,Hate
1,515476,1238239331879464960,1136405401770106880,0.999966,0.000625,0.000105,Hate
2,396712,1247813175996792834,1234197517006450688,0.999959,0.003865,0.000018,Hate
3,424027,1245317927823233024,1232772980763054081,0.999958,0.001476,0.000318,Hate
4,380359,1241373862756888577,907414928,0.999952,0.018423,0.000788,Hate


In [41]:
id_list = sorted_df.id
df1 = id_list.iloc[0:2000]
df1.columns = ['id']
df1 = df1.to_frame()
df1.head()

,id
0,1240976182494908416
1,1238239331879464960
2,1247813175996792834
3,1245317927823233024
4,1241373862756888577


In [42]:
hate_id = group_list(list(df1.id))
df_hate = tweets_request(hate_id)

100%|██████████| 20/20 [00:09<00:00,  2.20it/s]


In [60]:
df_hate = df_hate.reset_index(drop=True)
df_hate

,author_id,created_at,text,id
0,884055068265164801,2020-03-20T12:19:12.000Z,Fuck the ding dongs. Fuck the ching chongs. An...,1240976182494908416
1,1136405401770106880,2020-03-12T23:03:56.000Z,Fuck china fuck China fuck China\nFuck China f...,1238239331879464960
2,907414928,2020-03-21T14:39:27.000Z,"I do n’t believe this is eating bats, which is...",1241373862756888577
3,1134698470760017920,2020-03-13T22:13:08.000Z,"@Peoples_Pundit CCP Wuhan Virus, more Chinese ...",1238588936391290880
4,35525602,2020-03-19T22:56:27.000Z,Dr. Sean Lin said call it what it is - #ChinaV...,1240774162760617984
...,...,...,...,...
1002,1179030516973985794,2020-04-09T11:04:30.000Z,@jorge_guajardo Do not trust the CCP. They lie...,1248205139577430016
1003,79188351,2020-04-11T16:49:33.000Z,@MarkSimoneNY Quarantine China from the world ...,1249016750852771845
1004,1537476354,2020-03-30T13:23:34.000Z,@MarlowNYC BS - Wuhan Doctor Ai Fen has now di...,1244616256583618560
1005,1035477817092239362,2020-04-07T19:42:55.000Z,@wadewang85 @ccp_is_loser @IntyMedia @realDona...,1247610829773684744


In [61]:
pickle.dump(df_hate, open("../pickle/anti_asian_1000.pickle", "wb"))

# Database 2

In [58]:
df2 = pd.read_csv('../data/hate_add.csv')
df2.columns = ['id', 'label']
df2.head()

,id,label
0,572341498827522049,racism
1,572340476503724032,racism
2,572334712804384768,racism
3,572332655397629952,racism
4,575949086055997440,racism


In [48]:
df2.label.value_counts()

none      11559
sexism     3378
racism     1969
Name: label, dtype: int64

In [51]:
df_racsex = df2[(df2['label']=='racism') | (df2['label']=='sexism')]

In [55]:
racsex_id = group_list(list(df_racsex.id))
df_rac_sex = tweets_request(racsex_id)

100%|██████████| 54/54 [00:22<00:00,  2.42it/s]


In [62]:
df_rac_sex = df_rac_sex.reset_index(drop=True)
df_rac_sex

,id,author_id,created_at,text
0,572341498827522049,110114783,2015-03-02T10:23:41.000Z,Drasko they didn't cook half a bird you idiot ...
1,572340476503724032,38650214,2015-03-02T10:19:37.000Z,Hopefully someone cooks Drasko in the next ep ...
2,572334712804384768,2587278392,2015-03-02T09:56:43.000Z,of course you were born in serbia...you're as ...
3,572332655397629952,2601524623,2015-03-02T09:48:33.000Z,These girls are the equivalent of the irritati...
4,446460991396917248,930620467,2014-03-20T01:39:29.000Z,RT @YesYoureRacist: At least you're only a tin...
...,...,...,...,...
2736,569990621315919872,2756873076,2015-02-23T22:42:08.000Z,"RT @Superjutah: @MT8_9 £34,000 is £9000 above ..."
2737,570244879265206272,2756873076,2015-02-24T15:32:28.000Z,RT @DufresneFrank: “@gabystama: @DufresneFrank...
2738,571013698111860736,2756873076,2015-02-26T18:27:29.000Z,RT @BoycottBrandy: @ActionFlickDoc this person...
2739,571304517930774528,2756873076,2015-02-27T13:43:06.000Z,RT @Lyall: Patriarchy™ will expel me if I divu...


In [63]:
pickle.dump(df_rac_sex, open("../pickle/rac_sex_2741.pickle", "wb"))